In [ ]:
import pandas as pd
from pandas.io.json import json_normalize
import requests
import json
from urllib3.exceptions import InsecureRequestWarning
from urllib3 import disable_warnings
from fake_useragent import UserAgent
import time
import csv

In [ ]:
stock_company = pd.read_csv("/Users/elaine/Desktop/side_project/stock_price/company.csv",usecols=["main_stock_symbol","sub_stock_symbol","Company_E"],encoding='utf-8')

In [ ]:
sub_stock_symbol = stock_company["sub_stock_symbol"].to_list()
sub_stock_symbol

In [ ]:
#all
#date_year = ["2012","2013","2014","2015","2016","2017","2018","2019","2020","2021","2022"]
date_year = ["2017","2018","2019"]
date_month = ["0101","0201","0301","0401","0501","0601","0701","0801","0901","1001","1101","1201"]

stock_data_list = []
error_data_list = []

for stock_num in sub_stock_symbol:
    #print(stock_num)
    for year in date_year:
        #print(year)   
        for month in date_month:
            url = (f"https://www.twse.com.tw/rwd/zh/afterTrading/STOCK_DAY?date={year}{month}&stockNo={stock_num}&response=json&_=1679296642947")    
            ua = UserAgent()
            user_agent = ua.random
            headers = {'user-agent': user_agent}
            data = requests.get(url, headers=headers).text
            if data and "\"total\":0" in data:
                error = [str(year), str(month),str(stock_num),data]
                error_data_list.append(error)         
            else:
                stock_data_list.append(data)
        time.sleep(40)

In [ ]:
#save stock_data_list json file
with open("/Users/elaine/Desktop/side_project/stock_price/stock_price_2017_2019.json", "w") as outfile: 
    json.dump(stock_data_list, outfile)

In [ ]:
#save error_data_list csv file
with open('/Users/elaine/Desktop/side_project/stock_price/error_data_list_2017_2019.csv', 'w', newline='') as csvfile: 
    writer = csv.writer(csvfile)
    writer.writerow(["year","month","stock_number","raw_data"])
    writer.writerows(error_data_list)

In [ ]:
#csv to dataframe(error data)
error = pd.read_csv("/Users/elaine/Desktop/side_project/stock_price/error_data_list_2017_2019.csv", converters = {"month":str}) 
error